In [28]:
import numpy as np

In [29]:
import pandas as pd

from nltk import wordpunct_tokenize

In [31]:
from nltk import word_tokenize

In [49]:
news = pd.read_csv('estadao_noticias_eleicao.csv', sep=',', index_col='idNoticia')

In [50]:
news = news.sort_index(ascending=True)

In [51]:
news = news.replace(np.nan, '', regex=True)

# Converte o texto de cada celula para minusculo, e divide as frases em listas e as palavras em sublistas.

In [52]:
for i in range(1, len(news) + 1):
    for j in range(len(news.ix[i])):        
        news.ix[i][j] = str(news.ix[i][j]).lower() #força que a celula seja uma string, para uso da função lower()
        news.ix[i][j] = word_tokenize(news.ix[i][j])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


# Cria um índice invertido das palavras dos documentos, identificando em quantos e quais documentos a palavra aparece, e quantas vezes por documento:

# reference_list[word] = [doc_frequency,[[doc,word_frequency]]]

In [53]:
reference_list = {}

for i in range(1, len(news) + 1):
    for j in range(len(news.ix[i])):
        for k in news.ix[i][j]:
            if(reference_list.setdefault(k, None) == None):
                reference_list[k] = [1,[[i,1]]]
            elif(reference_list[k][1][-1][0] == i):
                reference_list[k][1][-1][1] += 1
            else:
                reference_list[k][0] += 1
                reference_list[k][1].append([i,1])

# FUNÇÕS DE BUSCA

In [36]:
def search_one_term(word):
    return list(reference_list[word])

In [37]:
def search_or_n_terms(words):
    result = reference_list[words[0]]
    
    for i in range(1, len(words)):
                result = np.union1d(result, reference_list[words[i]])
                # result é o resultado de sucessivas buscas AND das palavras
                
                return list(result)        

In [38]:
def smaller_terms(words):
    smaller = 0
    
    for i in range(1, len(words)):
            if (len(reference_list[words[smaller]]) > len(reference_list[words[i]])): 
                    smaller = i # posição da palavra com menos documentos
                    
    words[0], words[smaller] = words[smaller], words[0]
    #coloca o termo presente em menos documentos na posição inicial da lista
                    
    return None # função com efeito colateral

In [39]:
def search_and_n_terms(words):
    if (len(words) > 2):
        smaller = smaller_terms(words)
            
    result = reference_list[words[0]]
    
    for i in range(1, len(words)):
                result = np.intersect1d(result, reference_list[words[i]])
                # result é o resultado de sucessivas buscas OR das palavras
                
    return list(result)

# Função que trata e identifica a entrada, para selecionar a função de busca

In [40]:
def search(terms):
    words = terms.lower().split(" ")
    
    if(len(words) == 1):
        return search_one_term(words[0])
        
    if(words[1] == 'or'):
        return search_or_n_terms([words[0],words[2]])
    
    elif(words[1] == 'and'):
            return search_and_n_terms([words[0],words[2]])
        
    else:
        return search_and_n_terms(words)

# TESTES

In [44]:
news.ix[15][1][0]

"'pt"